In [57]:
from openai import OpenAI
import numpy as np

In [58]:
#client = OpenAI(api_key="sk-**************906sA")
client = OpenAI(api_key="sk-sv*************QvehX14nXoONvR0tKAQ8_XBfDFywvS3CVLyL4QaCm9Pklco6v0i3hhAA")
#client=OpenAI(api_key="sk-proj-CrqDvOF2JmZERQbFkLHBt1e5zjrA40JLXR****************1x-T3BlbkFJnLyP9JtRW1G3CO6iJnfTze3vgcYGaFWvfzkKxmQzXo--0TwqS6kREMvLUVUagYxx6xu1N3GSAA")
#client=OpenAI(api_key="sk-proj-GsflBnF1frso***********************KvbZtFNKIkzO-_T3BlbkFJMC755lcR-OiYrByH46JWIDd_9wqm64sB--SXDzNDwZ0fWt_Rv2ATUUjYmUHKPPnstEF8gKhxkA")
#===== Creating an embedding with the openai api =======
# https://platform.openai.com/docs/guides/embeddings

In [59]:
with open('messi.txt', 'r') as f:
    text = f.read()
    print(text)

Lionel Andrés Messi[note 1] (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ⓘ; born 24 June 1987), also known as Leo Messi, is an Argentine professional footballer who plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team. Widely regarded as one of the greatest players of all time, Messi set numerous records for individual accolades won throughout his professional footballing career such as eight Ballon d'Or awards and eight times being named the world's best player by FIFA.[note 2] He is the most decorated player in the history of professional football having won 45 team trophies,[note 3] including twelve league titles, four UEFA Champions Leagues, two Copa Américas, and one FIFA World Cup. Messi holds the records for most European Golden Shoes (6), most goals for a single club (672, with Barcelona), most goals (474), hat-tricks (36) and assists (192) in La Liga, most matches played (39), assists (18) and goal contributions (34) 

In [63]:
response = client.embeddings.create(
    input=text,
    model="text-embedding-3-small"
)
print(response.data[0].embedding)
# ======================================================

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [62]:
query = "Who is the greatest basketball player of all time?"

resource1 = "Lebron james is likely the greatest NBA player of all time."
resource2 = "Messi is likely the greatest soccer player of all time"
resource3 = "RAG stands for Retrieval Augmented Generation"

# Create a dictionary to hold sentence embeddings from messi.txt
messi_embeddings = {}

# Read the messi.txt file and create embeddings for chunks
with open('messi.txt', 'r') as file:
    text = file.read()
    # Split the text into chunks of 50 characters
    chunks = [text[i:i+50] for i in range(0, len(text), 50)]

    # Create embeddings for each chunk and store in the dictionary
    for chunk in chunks:
        embedding_response = client.embeddings.create(
            input=chunk,
            model="text-embedding-3-small"
        )
        embedding = embedding_response.data[0].embedding
        messi_embeddings[chunk] = embedding

query_embedding = client.embeddings.create(
        input=query,
        model="text-embedding-3-small"
    ).data[0].embedding

# Function to calculate cosine similarity
def cosine_simularity(A, B):
    return np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))

# Calculate cosine similarity for the query and each chunk
similarities = {}
for chunk, embedding in messi_embeddings.items():
    similarity = cosine_simularity(np.array(query_embedding), np.array(embedding))
    similarities[chunk] = similarity

# Get the top 5 chunks with the largest cosine similarity
top_chunks = sorted(similarities.items(), key=lambda item: item[1], reverse=True)[:5]

# Create a big string for the LLM prompt
top_sentences = " ".join([chunk for chunk, _ in top_chunks])
final_prompt = f"{query} {top_sentences}"

# Ask the LLM the answer to the question
answer_response = client.completions.create(
    prompt=final_prompt,
    model="text-davinci-003",  # Replace with your desired model
    max_tokens=100  # Adjust as necessary
)

# Print the response from the LLM
print("Response from LLM:", answer_response.choices[0].text.strip())



# Calculate similarity for predefined resources
resource_responses = client.embeddings.create(
            input=[resource1, resource2, resource3],
            model="text-embedding-3-small"
    ).data

resource1_embedding = resource_responses[0].embedding
resource2_embedding = resource_responses[1].embedding
resource3_embedding = resource_responses[2].embedding

query = np.array(query_embedding)

resource1_embedding = np.array(resource1_embedding)
resource2_embedding = np.array(resource2_embedding)
resource3_embedding = np.array(resource3_embedding)

simularity1 = cosine_simularity(query_embedding, resource1_embedding)
simularity2 = cosine_simularity(query_embedding, resource2_embedding)
simularity3 = cosine_simularity(query_embedding, resource3_embedding)

print(f"Simularity of query and resource1: {simularity1}")
print(f"Simularity of query and resource2: {simularity2}")
print(f"Simularity of query and resource3: {simularity3}")

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}